# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [2]:
#Your code here
import pandas as pd

data = pd.read_csv('homepage_actions.csv')

data.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [3]:
# Count the number of viewers (pageviews) and clickers (clicks)
num_viewers = data[data['action'] == 'view'].shape[0]
num_clickers = data[data['action'] == 'click'].shape[0]

print(f"Number of viewers: {num_viewers}")
print(f"Number of clickers: {num_clickers}")

Number of viewers: 6328
Number of clickers: 1860


In [4]:
# Find rows where someone clicked without viewing
click_without_view = data[(data['action'] == 'click') & (~data['id'].isin(data[data['action'] == 'view']['id']))]

click_without_view

,timestamp,id,group,action


## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [6]:
#Your code here
# Calculate the Click-Through Rate for control group
control_clicks = data[(data['group'] == 'control') & (data['action'] == 'click')].shape[0]
control_viewers = data[(data['group'] == 'control') & (data['action'] == 'view')].shape[0]
control_ctr = control_clicks / control_viewers

# Calculate the Click-Through Rate for control group
experiment_clicks = data[(data['group'] == 'experiment') & (data['action'] == 'click')].shape[0]
experiment_viewers = data[(data['group'] == 'experiment') & (data['action'] == 'view')].shape[0]
experiment_ctr = experiment_clicks / experiment_viewers

print(f"CTR for Control Group: {control_ctr:.4f}")
print(f"CTR for Experimental Group: {experiment_ctr:.4f}")

CTR for Control Group: 0.2797
CTR for Experimental Group: 0.3097


In [7]:
from statsmodels.stats.proportion import proportions_ztest

# Define the counts of successes (clicks) and the total number of trials (viewers) for each group
successes = [control_clicks, experiment_clicks]
trials = [control_viewers, experiment_viewers]

# Perform the two-sample proportion test
z_stat, p_value = proportions_ztest(successes, trials, alternative='two-sided')

print(f"Z-Statistic: {z_stat:.4f}")
print(f"P-Value: {p_value:.4f}")

Z-Statistic: -2.6186
P-Value: 0.0088


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [8]:
#Your code here
control_clicks = data[(data['group'] == 'control') & (data['action'] == 'click')].shape[0]
control_viewers = data[(data['group'] == 'control') & (data['action'] == 'view')].shape[0]
control_ctr = control_clicks / control_viewers

In [9]:
# Number of viewers in the experimental group
experiment_viewers = data[(data['group'] == 'experiment') & (data['action'] == 'view')].shape[0]

# Expected number of clicks in the experimental group
expected_clicks_experiment = control_ctr * experiment_viewers

print(f"Expected clicks for the experimental group: {expected_clicks_experiment:.2f}")

Expected clicks for the experimental group: 838.02


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [10]:
#Your code here
import math

# Calculate the standard error
standard_error = math.sqrt(experiment_viewers * control_ctr * (1 - control_ctr))

# Calculate the actual number of clicks in the experimental group
actual_clicks_experiment = experiment_clicks

# Calculate the z-score (number of standard deviations)
z_score = (actual_clicks_experiment - expected_clicks_experiment) / standard_error

print(f"Standard Error: {standard_error:.2f}")
print(f"Actual Clicks in Experimental Group: {actual_clicks_experiment}")
print(f"Z-Score: {z_score:.2f}")

Standard Error: 24.57
Actual Clicks in Experimental Group: 928
Z-Score: 3.66


### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [11]:
#Your code here
from scipy.stats import norm

# Calculate the p-value using the standard normal distribution
p_value = 2 * (1 - norm.cdf(abs(z_score)))

print(f"P-Value: {p_value:.4f}")

P-Value: 0.0002


### Analysis:

Does this result roughly match that of the previous statistical test?

> The result of the p-value calculation, which was derived by comparing the actual number of clicks in the experimental group to the expected number of clicks assuming the same click-through rate as the control group, is approximately 0.0002. This result aligns with the earlier statistical test, where we directly compared the click-through rates of the control and experimental groups using a two-sample proportion test, yielding a p-value of approximately 0.0088. In both tests, the p-values provide strong evidence to reject the null hypothesis. This suggests that the experimental homepage was more effective than the control group in terms of click-through rates. These results are statistically significant at conventional significance levels, indicating a substantial difference between the two groups.

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.